In [1]:
import numpy as np
import matplotlib.pyplot as plt

import gym

from gym import spaces


In [2]:
import ray
from ray import tune
from ray.rllib.examples.env.multi_agent import MultiAgentCartPole
from ray.rllib.examples.models.shared_weights_model import \
    SharedWeightsModel1, SharedWeightsModel2, TF2SharedWeightsModel, \
    TorchSharedWeightsModel
from ray.rllib.models import ModelCatalog
# from ray.rllib.policy import PolicySpec
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import register_env

from ray.rllib.env.multi_agent_env import MultiAgentEnv


import ray
from ray.rllib.agents.ppo import PPOTrainer, PPOTorchPolicy
from ray.rllib.agents.ppo import DEFAULT_CONFIG as DEFAULT_CONFIG_PPO

from ray.rllib.agents.dqn import DQNTrainer, DQNTorchPolicy
from ray.rllib.agents.dqn import  DEFAULT_CONFIG as DEFAULT_CONFIG_DQN


from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

ray.init(ignore_reinit_error=True, log_to_driver=False)

/home/peter/.local/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(
2021-08-10 14:47:36,342	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '192.168.1.21',
 'raylet_ip_address': '192.168.1.21',
 'redis_address': '192.168.1.21:64875',
 'object_store_address': '/tmp/ray/session_2021-08-10_14-47-34_494692_140440/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-08-10_14-47-34_494692_140440/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-08-10_14-47-34_494692_140440',
 'metrics_export_port': 59734,
 'node_id': '872ee5db35d6e39ca1fd3849a77853082d898391200b061fe1ac3a15'}

In [3]:
class MatrixGame():
    
    def __init__(self, RPST=(3,1,0,5)):
        
        self.RPST = RPST
        
        self.payoff_mat = np.empty((2,2), dtype=np.object)
        
        self.payoff_mat[0, 0] = (RPST[0], RPST[0])
        self.payoff_mat[1, 1] = (RPST[1], RPST[1])
        self.payoff_mat[0, 1] = (RPST[2], RPST[3])
        self.payoff_mat[1, 0] = (RPST[3], RPST[2])
        
    def play(self, a_row, a_col):
        # for ease of things 0 is coooperate
#                            1 is defect
        
        
#         if a_row == 'c':
#             row = 0
#         else:
#             row = 1
            
#         if a_col == 'c':
#             col = 0
#         else:
#             col = 1
            
        return self.payoff_mat[a_row, a_col]
        

In [4]:
class TwoAgentMatrixGameEnv(MultiAgentEnv):
    
    def __init__(self, RPST=(3,1,0,5), history_n=100):
        
        self.num_agents = 2
        
        self.RPST = RPST
        self.history_n = history_n
        self.history = np.zeros((2,2,self.history_n))
        
        self._counter = 0
        self._setup_spaces()
        self.game = MatrixGame(RPST=self.RPST)
    
    
    def _setup_spaces(self):
        
        self.action_space = spaces.Discrete(2)
        
        self.observation_space = spaces.Box(0, 1,
                                           shape=(self.history_n * 4,))
        
        
    def history_to_states(self, history=None):
        
        if history is None:
            history = self.history
            
        state1 = history.flatten()
        state2 = history[::-1,:,:].flatten()
        
        states = {0: state1, 1:state2}
        
        return states
            
        
        
    def step(self, action_dict):
        
        print((action_dict[0], action_dict[1]))
        rewards = self.game.play(action_dict[0], action_dict[1])
        rew = {i: rewards[i] for i in [0, 1]}
        
        self.history[0, action_dict[0], self._counter] = 1
        self.history[1, action_dict[1], self._counter] = 1
        
        obs = self.history_to_states(self.history)
        
        self._counter += 1
        
        is_done = self._counter >= self.history_n
        done = {i: is_done for i in [0, 1, "__all__"]}
        
        info = {0:{}, 1:{}}
        
        return obs, rew, done, info
        
        
        
    def reset(self):
        
        self.history = np.zeros((2,2,self.history_n))
        obs = self.history_to_states(self.history)

        self._counter = 0
        
        return obs
        

In [5]:
env = TwoAgentMatrixGameEnv()

In [7]:
aa = (4,3)
{i:aa[i] for i in [0,1]}

{0: 4, 1: 3}

In [8]:
env.step({0:0,1:0})

(0, 0)


({0: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [9]:
info ={}

In [10]:
info[0] = {}

In [11]:
info[1] = {}

In [12]:
info

{0: {}, 1: {}}

In [13]:
register_env('twoagent_PD', lambda c: TwoAgentMatrixGameEnv())

In [14]:
trainer_config_ppo = DEFAULT_CONFIG_PPO.copy()
trainer_config_ppo['num_workers'] = 1
trainer_config_ppo['num_sgd_iter'] = 20
trainer_config_ppo['sgd_minibatch_size'] = 32
# trainer_config_ppo['model']['fcnet_hiddens'] = [1024, 512,512, 256,256,32,8]
trainer_config_ppo['model']['fcnet_hiddens'] = [256,256,32,8]

trainer_config_ppo['num_cpus_per_worker'] = 0

In [15]:
trainer = PPOTrainer(trainer_config_ppo, env="twoagent_PD");


2021-08-10 14:44:16,235	INFO trainer.py:671 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-08-10 14:44:16,237	INFO trainer.py:696 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


KeyboardInterrupt: 

In [ ]:
for i in range(2):
    print("Training iteration {}...".format(i))
    result=trainer.train()
    print(pretty_print(result))

In [ ]:
env = TwoAgentMatrixGameEnv()

In [1]:

n_samples = 50


defects = []
rewards = []
for i in range(n_samples):
    state = env.reset()
    
    total_defect = 0
    cum_reward = 0
    done = False
    i = 0
    while not done:
        print('===========')
        print(i)
        print('===========')
        
        i+=1
        action = trainer.compute_actions(state)
        print(action)
#         total_defect += action
        state, reward, done, results = env.step(action)
#         cum_reward += reward
    defects.append(total_defect)
    rewards.append(cum_reward)

NameError: name 'env' is not defined

In [6]:

def env_creator(_):
    return TwoAgentMatrixGameEnv()
single_env = TwoAgentMatrixGameEnv()
env_name = "TwoAgent_PD"
register_env(env_name, env_creator)


obs_space = single_env.observation_space
act_space = single_env.action_space
num_agents = single_env.num_agents

def gen_policy():
    return (PPOTorchPolicy, obs_space, act_space, {})
policy_graphs = {}

for i in range(num_agents):
    policy_graphs['agent-' + str(i)] = gen_policy()
def policy_mapping_fn(agent_id):
        return 'agent-' + str(agent_id)

In [7]:
config={
    "log_level": "WARN",
    "num_workers": 3,
    "num_cpus_for_driver": 1,
    "num_cpus_per_worker": 1,
    "lr": 5e-3,
    "model":{"fcnet_hiddens": [1024, 512,256,32,8]},
    "multiagent": {
        "policies": policy_graphs,
        "policy_mapping_fn": policy_mapping_fn,
    },
    "env": "TwoAgent_PD"
}

In [8]:
exp_name = 'TA_TEST1'
exp_dict = {
        'name': exp_name,
        'run_or_experiment': 'PPO',
        "stop": {
            "training_iteration": 20
        },
        'checkpoint_freq': 20,
        "config": config,
}
# ray.init()
tune.run(**exp_dict)

Trial name,status,loc
PPO_TwoAgent_PD_57fc6_00000,PENDING,


Result for PPO_TwoAgent_PD_57fc6_00000:
  agent_timesteps_total: 15996
  custom_metrics: {}
  date: 2021-08-10_14-52-20
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 481.0
  episode_reward_mean: 449.79487179487177
  episode_reward_min: 417.0
  episodes_this_iter: 78
  episodes_total: 78
  experiment_id: b8bdad5b0cf241c18137debbeebc8113
  hostname: coolo-computer
  info:
    learner:
      agent-0:
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 0.1999999999999999
          cur_lr: 0.005000000000000001
          entropy: 0.67753576570087
          entropy_coeff: 0.0
          kl: 0.016009656919373408
          policy_loss: -0.005544625104419769
          total_loss: 6696.291372147817
          vf_explained_var: -8.514949634275126e-09
          vf_loss: 6696.293759300595
      agent-1:
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 0.1999999999999999
          cur_lr: 0.005000000000000001

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_TwoAgent_PD_57fc6_00000,RUNNING,192.168.1.21:140555,1,263.047,7998,449.795,481,417,100


2021-08-10 14:54:58,545	WARNING tune.py:506 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Result for PPO_TwoAgent_PD_57fc6_00000:
  agent_timesteps_total: 31992
  custom_metrics: {}
  date: 2021-08-10_14-56-33
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 482.0
  episode_reward_mean: 447.57
  episode_reward_min: 422.0
  episodes_this_iter: 81
  episodes_total: 159
  experiment_id: b8bdad5b0cf241c18137debbeebc8113
  hostname: coolo-computer
  info:
    learner:
      agent-0:
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 0.1999999999999999
          cur_lr: 0.005000000000000001
          entropy: 0.6699515495981488
          entropy_coeff: 0.0
          kl: 0.00733576962988179
          policy_loss: 0.0003143421684702237
          total_loss: 7493.894639756944
          vf_explained_var: -4.730527525254047e-09
          vf_loss: 7493.892973400298
      agent-1:
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 0.09999999999999995
          cur_lr: 0.005000000000000001
        

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_TwoAgent_PD_57fc6_00000,RUNNING,192.168.1.21:140555,2,516.19,15996,447.57,482,422,100


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_TwoAgent_PD_57fc6_00000,RUNNING,192.168.1.21:140555,2,516.19,15996,447.57,482,422,100


2021-08-10 14:56:33,874	ERROR tune.py:545 -- Trials did not complete: [PPO_TwoAgent_PD_57fc6_00000]
2021-08-10 14:56:33,875	INFO tune.py:549 -- Total run time: 527.22 seconds (526.85 seconds for the tuning loop).
2021-08-10 14:56:33,876	WARNING tune.py:553 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


In [ ]:
tra